In [36]:
import os
import json

import pandas as pd

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [37]:
%run "../scorer.py"

In [38]:
credential_loc = "../../credentials.json"

data_type = "training"
corpus = "The Telegraph"

base_loc = f"/Volumes/BCross/datasets/author_verification/"
data_loc = f"{base_loc}{data_type}/{corpus}/"

raw_data_loc = f"{data_loc}known_raw.jsonl"
processed_data_loc = f"{data_loc}known_processed.jsonl"
batch_complete_loc = f"{data_loc}batch_sentence_complete/"
post_process_loc = f"{data_loc}batch_postprocessed/"

# ParaScore save location
parascore_loc = f"{data_loc}batch_parascore/"
os.makedirs(parascore_loc, exist_ok=True)

# Phone number for WhatsApp notifications
phone_number = "+447756976114"

In [45]:
raw_df = read_jsonl(raw_data_loc)

batch_completed_files = [
    f for f in os.listdir(batch_complete_loc)
    if os.path.isfile(os.path.join(batch_complete_loc, f)) and f.endswith('.jsonl')
]

files_postprocessed = [
    f for f in os.listdir(post_process_loc)
    if os.path.isfile(os.path.join(post_process_loc, f)) and f.endswith('.jsonl')
]

# Replace "batch_" with "doc_" in each element of files_processed
# files_postprocessed = [file.replace("doc_", "batch_") for file in files_postprocessed]

files_parascore_complete = [
    f for f in os.listdir(parascore_loc)
    if os.path.isfile(os.path.join(parascore_loc, f)) and f.endswith('.jsonl')
]

files_to_be_processed = list(set(files_postprocessed) - set(files_parascore_complete))
files_to_be_processed = sorted(files_to_be_processed)

print(f"Number of documents to process in raw data: {len(raw_df['doc_id'])}")
print(f"Files Complete in Batch location: {len(batch_completed_files)}")
print(f"Files Post Processed: {len(files_postprocessed)}")
print(f"Files with Score: {len(files_parascore_complete)}")
print(f"Files to be Processed: {len(files_to_be_processed)}")

Number of documents to process in raw data: 220
Files Complete in Batch location: 220
Files Post Processed: 205
Files with Score: 133
Files to be Processed: 72


In [46]:
parascore_free = ParaphraseScorer(score_type='parascore_free', model_type='bert-base-uncased')

In [49]:
for idx, file in enumerate(files_to_be_processed):
    
    print(f"Processing {idx + 1} out of {len(files_to_be_processed)}: {os.path.basename(file)}")
    output_file_loc = f"{parascore_loc}{file}"
    
    try:
        
        df = read_jsonl(f"{post_process_loc}{file}")

        if 'original' not in df.columns:
            df.rename(columns={"original_sentence": "original"}, inplace=True)

        df_with_score = parascore_free.calculate_score(df)

        write_jsonl(df_with_score, output_file_loc)
        
    except Exception as e:
        print(f"Error processing {file}: {e}")
        continue

Processing 1 out of 72: doc_alicevincent_text_3.jsonl
Processing 2 out of 72: doc_allisonpearson_text_1.jsonl
Processing 3 out of 72: doc_allisonpearson_text_2.jsonl
Processing 4 out of 72: doc_ambroseevans_pritchard_text_2.jsonl
Processing 5 out of 72: doc_andrewenglish_text_3.jsonl
Processing 6 out of 72: doc_andrewmarszal_text_2.jsonl
Processing 7 out of 72: doc_annatyzack_text_1.jsonl
Processing 8 out of 72: doc_anthonyperegrine_text_1.jsonl
Processing 9 out of 72: doc_beewilson_text_3.jsonl
Processing 10 out of 72: doc_benfarmer_text_3.jsonl
Processing 11 out of 72: doc_benfogle_text_2.jsonl
Processing 12 out of 72: doc_benfogle_text_3.jsonl
Processing 13 out of 72: doc_benlawrence_text_1.jsonl
Processing 14 out of 72: doc_benlawrence_text_2.jsonl
Processing 15 out of 72: doc_benmarlow_text_1.jsonl
Processing 16 out of 72: doc_benmarlow_text_2.jsonl
Processing 17 out of 72: doc_bernadettemcnulty_text_2.jsonl
Processing 18 out of 72: doc_beverleyturner_text_1.jsonl
Processing 19 ou